# 1. Prueba de módulo "webscraping"

**scrap_discursos:** Scrap de discursos desde una web paginada con links individuales por discurso.

Parámetros:
- base_url (str): URL de inicio.
- xpaths (dict): Diccionario con claves: 'link_items', 'boton_siguiente', 'titulo', 'fecha', 'contenido'.
- espera (int): Tiempo de espera entre interacciones.
- paginas (int): Número máximo de páginas a recorrer.
- articulos_maximos (int): Límite de discursos a scrapear.
- headless (bool): Si True, navegador sin interfaz.
- verbose (bool): Si True, muestra estado.
- output_path (str): Ruta opcional para guardar CSV.

Retorna:
- DataFrame con columnas: 'url', 'titulo', 'fecha', 'contenido', 'codigo'

In [7]:
from modulos.webscraping import xpaths_casarosada, scrap_discursos
import modulos.paths as paths

df = scrap_discursos(
    base_url="https://www.casarosada.gob.ar/informacion/discursos",
    xpaths=xpaths_casarosada,
    espera=3,
    paginas=5,
    articulos_maximos=1,
    headless=True,
    verbose=True,
    output_path=paths.discursos
)

df.head()

🌐 Página 1
➕ 40 nuevos links

🔗 Total de links únicos obtenidos: 1

📄 (1/1) Procesando: https://www.casarosada.gob.ar/informacion/discursos/51036-palabras-del-presidente-de-la-nacion-javier-milei-en-la-derecha-fest-cordoba
📝 Palabras del Presidente de la Nación, Javier Milei, en La Derecha Fest, Córdoba (Martes 22 de julio de 2025) - 6640 palabras

✅ Archivo 'C:\PROYECTOS\Primer MVP1\data\discursos.csv' guardado correctamente.


,url,titulo,fecha,contenido,codigo
0,https://www.casarosada.gob.ar/informacion/disc...,"Palabras del Presidente de la Nación, Javier M...",Martes 22 de julio de 2025,"Palabras del Presidente de la Nación, Javier M...",DISCURSO_001


# 2. Prueba de módulo "preprocesamiento"

## 2.1. Segmentación en recortes

**generar_recortes:** Convierte una base de discursos en una base de recortes frase a frase.

Parámetros:
- df_discursos (DataFrame): Debe tener columnas 'titulo' y 'contenido'.
- agregar_codigo (bool): Si True, genera columna 'codigo' con formato DISCURSO_001...
- prefijo_codigo (str): Prefijo para los códigos asignados.
- guardar (bool): Si True, guarda la base generada como CSV.
- output_path (str): Ruta del archivo CSV de salida (requerido si guardar=True).

Retorna:
- DataFrame con columnas: 'codigo', 'recorte_id', 'posicion', 'frase'.

In [8]:
import pandas as pd
import modulos.paths as paths
from modulos.preprocesamiento import generar_recortes

# Cargar discursos scrapeados
df_discursos = pd.read_csv(paths.discursos, encoding="utf-8-sig")

# Generar recortes y guardar
df_recortes = generar_recortes(df_discursos, guardar=True, output_path=paths.recortes)

✅ Archivo CSV generado correctamente.
📄 Ruta: C:\PROYECTOS\Primer MVP1\data\recortes.csv
🧾 La base tiene 242 observaciones (frases).


## 2.2. Filtrado de discursos por cantidad de frases (mínimo: 24, para series temporales)

**filtrar_discursos:** Filtra los códigos que tienen al menos `umbral` frases en `df_recortes`, y opcionalmente guarda los resultados si se indican los paths.

Parámetros:
- df: DataFrame original con los textos completos.
- df_recortes: DataFrame con las frases (recortes) y la columna 'codigo'.
- umbral: Cantidad mínima de frases necesarias para conservar un código.
- guardar: Si True, guarda los DataFrames filtrados y la lista de códigos eliminados.
- path_discursos: Ruta para guardar el CSV de discursos filtrados.
- path_recortes: Ruta para guardar el CSV de recortes filtrados.
- path_codigos_eliminados: Ruta para guardar el TXT con los códigos eliminados.

Retorna:
- df_filtrado: DataFrame con discursos filtrados.
- df_recortes_filtrado: DataFrame de recortes correspondientes a los discursos filtrados.
- codigos_eliminados: lista de códigos de discursos eliminados.
- codigos_validos : lista de códigos de discursos válidos.
- conteo_frases: DataFrame con el conteo total de frases por código, antes del filtrado.

In [9]:
import pandas as pd
import modulos.paths as paths
from modulos.preprocesamiento import filtrar_discursos

df = pd.read_csv(paths.discursos, encoding="utf-8-sig")
df_recortes = pd.read_csv(paths.recortes, encoding="utf-8-sig")

df_filtrado, df_recortes_filtrado, codigos_eliminados, codigos_validos, conteo_frases = filtrar_discursos(
    df=df,
    df_recortes=df_recortes,
    umbral=24,
    guardar=True,
    path_discursos=paths.discursos_filtrado,
    path_recortes=paths.recortes_filtrado,
    path_codigos_eliminados=paths.codigos_eliminados
)

Cantidad de frases por código (primeras filas):
         codigo  cantidad_frases
0  DISCURSO_001              242

✅ Códigos con al menos 24 frases: 1
❌ Códigos eliminados (menos de 24 frases): 0

📄 Textos originales tras el filtro: 1
🧾 Frases tras el filtro: 242

💾 Archivos guardados:
- Discursos: C:\PROYECTOS\Primer MVP1\data\discursos_filtrado.csv
- Recortes: C:\PROYECTOS\Primer MVP1\data\recortes_filtrado.csv
- Códigos eliminados: C:\PROYECTOS\Primer MVP1\data\codigos_eliminados.txt


## 2.3. Limpieza y preprocesamiento

**procesar_textos:** Procesa textos en una columna de un DataFrame aplicando limpieza y análisis lingüístico.

Parámetros:
- df (pd.DataFrame): DataFrame original.
- columna_texto (str): Nombre de la columna con texto.
- texto_limpio (bool): Si se debe incluir columna con texto limpio.
- tokens (bool): Si se deben incluir los tokens.
- lemmas (bool): Si se deben incluir los lemas.
- pos_tags (bool): Si se deben incluir etiquetas POS.
- dependencias (bool): Si se deben incluir dependencias sintácticas.
- entidades (bool): Si se deben incluir entidades nombradas.
- sujetos (bool): Si se debe marcar sujeto omitido por frase.
- guardar (bool): Si se debe guardar el resultado a CSV.
- path_salida (str): Ruta de salida (obligatoria si guardar=True).

Retorna:
- pd.DataFrame: DataFrame con columnas nuevas según los flags seleccionados.

In [5]:
import pandas as pd
import modulos.paths as paths
from modulos.preprocesamiento import procesar_textos

df_filtrado = pd.read_csv(paths.discursos_filtrado, encoding="utf-8-sig")

df_resultado = procesar_textos(
    df=df_filtrado,
    columna_texto="contenido",
    texto_limpio=True,
    tokens=False,
    lemmas=False,
    pos_tags=False,
    dependencias=False,
    entidades=False,
    sujetos=False,
    guardar=True,
    path_salida=paths.discursos_preprocesado
)

✅ Archivo guardado: C:\PROYECTOS\Primer MVP1\data\discursos_preprocesado.csv


In [6]:
df_recortes_filtrado = pd.read_csv(paths.recortes_filtrado, encoding="utf-8-sig")

df_resultado = procesar_textos(
    df=df_recortes_filtrado,
    columna_texto="frase",
    texto_limpio=True,
    tokens=True,
    lemmas=True,
    pos_tags=True,
    dependencias=True,
    entidades=True,
    sujetos=True,
    guardar=True,
    path_salida=paths.recortes_preprocesado
)

✅ Archivo guardado: C:\PROYECTOS\Primer MVP1\data\recortes_preprocesado.csv


**Nota para optimización:** Ver de que no se dupliquen entidades y dependencias en el archivo

# 3. Prueba de módulo "resumen"

Este módulo permite generar resúmenes de discursos extensos utilizando un LLM, a través de un enfoque por bloques temáticos. La función principal aplicada es resumir_dataframe.

Lógica del procedimiento
- Segmentación semántica: el texto se divide en fragmentos según cambios temáticos detectados.
- Resumen parcial: cada fragmento se resume individualmente mediante un LLM.
- Redacción global: se produce un resumen final fluido e integrado a partir de los resúmenes parciales.

**resumir_dataframe:** Genera resúmenes utilizando un LLM para cada fila de un DataFrame que contenga la columna texto_limpio. Utiliza dos tipos de prompts: uno para fragmentos y otro para la redacción final del discurso completo.

Parámetros:
- df (pd.DataFrame): DataFrame con columnas 'codigo', 'titulo', 'fecha', 'texto_limpio'.
- modelo_llm (Callable[[str], str]): Función que toma un prompt (str) y devuelve la respuesta generada por el modelo LLM.
- prompt_fragmento (str): Prompt utilizado para resumir cada fragmento segmentado del discurso.
- prompt_discurso (str): Prompt utilizado para redactar el resumen global del discurso a partir de los resúmenes parciales.
- umbral (float): Umbral de sensibilidad para la segmentación temática. Valores más bajos generan más fragmentos.
- guardar (bool): Si es True, guarda el DataFrame resultante en un archivo .csv.
- path_salida (str): Ruta del archivo de salida si guardar=True.
- mostrar_prompts (bool): Si es True, imprime los prompts utilizados durante la ejecución (útil para debugging).

Retorna:
- pd.DataFrame: DataFrame con una nueva columna 'resumen' que contiene el resumen generado por LLM para cada discurso.

In [1]:
import pandas as pd
import modulos.paths as paths
import modulos.prompts as prompts
from modulos.resumen import resumir_dataframe
from modulos.modelo import get_model

modelo_llm = get_model(modelo="mistral")

df_preprocesado = pd.read_csv(paths.discursos_preprocesado, encoding="utf-8-sig")

df_con_resumenes = resumir_dataframe(
    df=df_preprocesado,
    modelo_llm=modelo_llm,
    prompt_fragmento=prompts.PROMPT_RESUMIR_FRAGMENTO,
    prompt_discurso=prompts.PROMPT_RESUMIR_DISCURSO,
    umbral=0.25,
    guardar=True,
    path_salida=paths.discursos_resumen,
    mostrar_prompts=True
)

C:\ProgramData\anaconda3\envs\ag_env2\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
C:\ProgramData\anaconda3\envs\ag_env2\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: distiluse-base-multilingual-cased-v1
C:\ProgramData\anaconda3\envs\ag_env2\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  w

Batches:   0%|          | 0/2 [00:00<?, ?it/s]


📤 Prompt fragmento:
Resumí en español el siguiente fragmento de un discurso político, manteniendo su sentido general, tono e ideas principales. No inventes información y usá lenguaje claro y preciso:
hola a todos. che, ¿no van a sacar? ¿cómo es? ¿saquen al pingüino del cajón? ¡viva la libertad, carajo!



INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"



📤 Prompt fragmento:
Resumí en español el siguiente fragmento de un discurso político, manteniendo su sentido general, tono e ideas principales. No inventes información y usá lenguaje claro y preciso:
primero que nada, quiero extenderle un cálido saludo a mi queridísimo amigo agustín laje como así también a todo el equipo de la derecha diario por organizar este enorme evento. muchas gracias. también quiero agradecer a todos los expositores que me precedieron y, por supuesto, a todos ustedes. y también quiero agradecer a todos aquellos que no pudieron venir, pero nos están siguiendo desde sus casas. son ustedes que dan todo por sacar adelante un país al que años de estatismo descontrolado parecían haber condenado al país a la miseria. son ustedes quienes vieron a sus amigos armar valijas y emigrar, quienes vieron a sus padres sufrir la inflación y las devaluaciones, quienes vieron cómo cada día el sueño y la independencia económica y el progreso se alejaba cada vez más y más. muchachos:

...


📤 Prompt fragmento:
Resumí en español el siguiente fragmento de un discurso político, manteniendo su sentido general, tono e ideas principales. No inventes información y usá lenguaje claro y preciso:
pero para lograrlo no podemos descuidarnos ni un solo segundo. la casta política sigue ahí, esperando que nos equivoquemos. se prepararon toda su vida sirviéndose del estado y saltando de cargo en cargo. conocen a la perfección cada engranaje de la máquina de impedir. máquina que ellos mismos construyeron para evitar los cambios que la sociedad demanda. nos quieren ver débiles para poder llevarnos puestos y así retomar el poder, con el único propósito de recuperar los privilegios que les estamos arrebatando en pos de devolverle la libertad a cada argentino de bien. no hay que permitírselo. es el momento de estar más unidos que nunca, fieles a nuestro propósito. ellos quieren que seamos un país de pobres y sometidos. nosotros queremos un país próspero y libre. esta es la consigna que debem

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"



📤 Prompt final:
Título del discurso: Palabras del Presidente de la Nación, Javier Milei, en La Derecha Fest, Córdoba
Fecha: Martes 22 de julio de 2025
A continuación tenés los resúmenes parciales de un discurso político extenso.
Redactá un resumen final fluido y preciso, sin repetir ideas, manteniendo el tono original y resaltando los temas clave del discurso.
Evitá generalidades vacías o fórmulas genéricas. Escribí en español claro:
Hola a todos. ¿No vais a salir? ¿Qué pasa? ¿Váyanse el pingüino del armario? ¡Viva la libertad, carajo!
Resumen: El orador habla con una expresión coloquial para preguntar a la audiencia si está listo para salir de algún lugar. Luego hace una pregunta absurda para despertar la atención y termina llamando a la libertad en un tono entusiasta. La idea principal es que el público esté listo para salir y estar ansioso por la libertad.
The text appears to be discussing the concepts and critiques of certain ideologies, such as neoliberalism and social equality, 

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
Generando resúmenes con LLM: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [17:50<00:00, 1070.76s/it]


🟩 Resumen generado con éxito para código: DISCURSO_001
Resumen:
En este discurso político, Javier Milei aborda temas como el liberalismo y el colectivismo, criticando la promoción de diversidad, equidad e inclusión en empresas y sociedad en general. Según el orador, esto ha sustituido la igualdad ante la ley por discriminación positiva a grupos sociales, lo que fragmenta la sociedad y debilita las instituciones. El discurso se enfoca en llamar a la libertad, afirmando que Argentina necesita una nueva dirección política para sacarla de su último siglo de humillación y llevar sus ideas de libertad al gran público. Menciona la necesidad de una batalla cultural por las ideas de libertad y progreso, enfatizando la importancia de ser soldados en esta lucha. Además, hace un llamado a Argentina para ser el país más grande de América Latina y exhorta a los oyentes a sentirse orgullosos de ser argentinos nuevamente. El discurso enfatiza la necesidad de un cambio en la política argentina y una b

# 4. Prueba de módulo "identificacion_actores"

## 4.1. Identificación de tipo de discurso, enunciador, enunciatarios y lugar

**procesar_discursos_llm:** Procesa un conjunto de discursos para identificar automáticamente su tipo, enunciador, enunciatarios y lugar, utilizando un modelo de lenguaje (LLM) a través de la API de Ollama y prompts personalizados.

Parámetros:
- df (pd.DataFrame): DataFrame con columnas 'codigo', 'titulo', 'texto_limpio' y 'resumen'.
- diccionario (dict): Diccionario conceptual con categorías y ejemplos, utilizado para orientar la identificación de tipo de discurso y roles enunciativos (ej. diccionario_tipos_discurso).
- prompt_tipo (str): Prompt base para la detección del tipo de discurso. Debe incluir los placeholders "RESUMEN", "FRAGMENTOS" y "DICCIONARIO".
- prompt_enunciacion (str): Prompt base para la identificación del enunciador y los enunciatarios. Debe incluir los placeholders "RESUMEN", "FRAGMENTOS" y "DICCIONARIO".
- prompt_lugar (str): Prompt base para la detección del lugar de enunciación. Debe incluir los placeholders "TITULO", "RESUMEN" y "FRAGMENTOS".
- guardar_csv (bool, opcional): Si es True, guarda los resultados en un archivo .csv. Por defecto: True.
- output_path (str, opcional): Ruta al archivo donde se guardará el CSV. Obligatorio si guardar_csv=True.
- analizar_tipo (bool, opcional): Si es True, se realiza la identificación del tipo de discurso. Por defecto: True.
- analizar_enunc (bool, opcional): Si es True, se identifican enunciador y enunciatarios. Por defecto: True.
- analizar_lug (bool, opcional): Si es True, se identifica el lugar de enunciación. Por defecto: True.
- mostrar_prompts (bool, opcional): Si es True, imprime los prompts construidos antes de enviarlos al modelo. Útil para debugging. Por defecto: False.

Retorna:
- pd.DataFrame: DataFrame original con columnas adicionales que contienen los resultados de la identificación automática (tipo_discurso, tipo_discurso_justificacion, enunciador_actor, enunciador_justificacion, enunciatario_0_actor, enunciatario_0_tipo, enunciatario_0_justificacion, ..., lugar_ciudad, lugar_provincia, lugar_pais, lugar_justificacion).

Requiere:
- Definición previa de los paths en el módulo modulos.paths.
- Disponibilidad de los prompts base en el módulo modulos.prompts.
- Diccionario conceptual diccionario_tipos_discurso importado desde modulos.tipos_discurso.

In [1]:
# Cargar dataset con resúmenes
import pandas as pd
import modulos.paths as paths

df_discursos = pd.read_csv(paths.discursos_resumen, encoding="utf-8-sig")

# Procesar discursos con el LLM
import modulos.prompts as prompts
from modulos.tipos_discurso import diccionario_tipos_discurso
from modulos.enunciacion import procesar_discursos_llm
from modulos.modelo import llm
df_resultado = procesar_discursos_llm(
    df=df_discursos,
    modelo_llm=llm,
    diccionario=diccionario_tipos_discurso,
    prompt_tipo=prompts.PROMPT_TIPO_DISCURSO,
    prompt_enunciacion=prompts.PROMPT_ENUNCIACION,
    prompt_lugar=prompts.PROMPT_LUGAR,
    guardar_csv=True,
    output_path=paths.discursos_enunc,
    analizar_tipo=True,
    analizar_enunc=True,
    analizar_lug=True,
    mostrar_prompts=True
)

C:\ProgramData\anaconda3\envs\ag_env2\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
Procesando discursos:   0%|                                                                                                                                                                                           | 0/1 [00:00<?, ?it/s]


📤 Prompt - Tipo de discurso:
 Estás analizando un discurso. Tu única tarea es identificar el tipo de discurso y justificar por qué.
⚠️ Siempre agregá una justificación sobre por qué identificaste el discurso como correspondiente a un determinado tipo.
### Resumen del discurso:
En este discurso político, Javier Milei aborda temas como el liberalismo y el colectivismo, criticando la promoción de diversidad, equidad e inclusión en empresas y sociedad en general. Según el orador, esto ha sustituido la igualdad ante la ley por discriminación positiva a grupos sociales, lo que fragmenta la sociedad y debilita las instituciones. El discurso se enfoca en llamar a la libertad, afirmando que Argentina necesita una nueva dirección política para sacarla de su último siglo de humillación y llevar sus ideas de libertad al gran público. Menciona la necesidad de una batalla cultural por las ideas de libertad y progreso, enfatizando la importancia de ser soldados en esta lucha. Además, hace un llamado

Procesando discursos: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [02:49<00:00, 169.51s/it]


🎤 Código: DISCURSO_001
✔️ Tipo de discurso: discurso político
✔️ Enunciador: Javier Milei
📨 Enunciatarios:
   1. La derecha (prodestinatario)
   2. La oposición política explícita (contradestinatario)
   3. La población argentina en general (paradestinatario)
✔️ Lugar: Córdoba Córdoba Argentina

✅ Archivo guardado como C:\PROYECTOS\Primer MVP1\data\A5. discursos_enunciacion.csv


## 4.2. Identificación de actores con contexto enunciativo

**identificar_actores_con_contexto:** Procesa una serie de recortes textuales para identificar actores relevantes mencionados en cada frase, excluyendo al enunciador y los enunciatarios previamente detectados. Para ello, utiliza un modelo de lenguaje (LLM) mediante la API de Ollama y un prompt estructurado que combina información del discurso de inscripción (resumen, fecha, lugar, tipo, enunciador y enunciatarios) con la frase objetivo y su contexto.

Parámetros:
- df_recortes (pd.DataFrame): DataFrame que contiene los recortes textuales a analizar. Debe incluir al menos las columnas 'frase', 'recorte_id' y 'codigo' (identificador del discurso de inscripción).
- df_enunc (pd.DataFrame): DataFrame con los discursos completos y los resultados de la identificación enunciativa (enunciador, enunciatarios, tipo de discurso, lugar, etc.), asociados por la columna 'codigo'.
- prompt_template (str): Prompt base con placeholders que será completado dinámicamente para cada recorte. Debe incluir:
- a. {resumen_global}: Resumen del discurso de inscripción.
- b. {fecha}: Fecha del discurso.
- c. {lugar_justificacion}: Lugar del discurso (con justificación).
- d. {tipo_discurso}: Tipo de discurso identificado.
- e. {enunciador}: Enunciador del discurso.
- f. {enunciatarios}: Enunciatarios identificados.
- g. {frase}: Frase objetivo.
- h. {frases_contexto}: Frases inmediatamente anteriores y posteriores a la frase objetivo.
- i. {heuristicas}: Lista de reglas de inferencia válidas.
- j. {ontologia}: Ontología de actores posibles (categorías permitidas para clasificación).
- path_errores (str): Ruta del archivo donde se guardarán los errores o casos que no puedan procesarse correctamente.
- output_path (str): Ruta del archivo donde se guardarán los resultados procesados en formato .csv.
- modelo_llm (callable): Función o clase del modelo LLM configurado, que recibe el prompt como string y devuelve una respuesta.
- mostrar_prompts (bool, opcional): Si es True, imprime el prompt generado para cada fragmento antes de enviarlo al modelo. Útil para debugging. Por defecto: False.
- guardar_resultados (bool, opcional): Si es True, guarda el DataFrame resultante como CSV en output_path. Por defecto: True.

Retorna:
- pd.DataFrame: DataFrame original con columnas adicionales que contienen los actores identificados por frase (actor, tipo, modo, regla de inferencia), respetando el formato JSON indicado.

Requiere:
- Definición previa de los paths relevantes en el módulo modulos.paths.
- Prompt base definido en modulos.prompts.PROMPT_IDENTIFICAR_ACTORES.
- Preprocesamiento de enunciadores y enunciatarios mediante la función procesar_discursos_llm (u otra que genere df_enunc compatible).

In [1]:
import pandas as pd
import modulos.paths as paths
import modulos.prompts as prompts
from modulos.identificacion_actores import identificar_actores_con_contexto
from modulos.modelo import llm

df_recortes = pd.read_csv(paths.recortes_prueba, encoding="utf-8-sig")
df_enunc = pd.read_csv(paths.discursos_enunc, encoding="utf-8-sig")

df_resultado = identificar_actores_con_contexto(
    df_recortes=df_recortes,
    df_enunc=df_enunc,
    prompt_template=prompts.PROMPT_IDENTIFICAR_ACTORES,
    path_errores=paths.errores_identificacion_actores,
    output_path=paths.actores_identificados,
    modelo_llm=llm,
    mostrar_prompts=False,
    guardar_resultados=True
)

C:\ProgramData\anaconda3\envs\ag_env2\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
  0%|                                                                                                                                                                                                                | 0/10 [00:00<?, ?it/s]C:\PROYECTOS\Primer MVP1\modulos\identificacion_actores.py:98: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  respuesta = modelo_llm([HumanMessage(content=prompt)]).content
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [06:01<00:00, 36.11s/it]


In [2]:
df_actores = pd.read_csv("data/C1. actores_identificados.csv", encoding="utf-8-sig")

df_actores.head(20)

,actor,tipo,modo,frase_idx,recorte_id,codigo,regla_de_inferencia,regla de inferencia
0,audiencia,colectivo,explícito,0,DISCURSO_001_FR_001,DISCURSO_001,NaN,NaN
1,audiencia,colectivo,inferido,1,DISCURSO_001_FR_002,DISCURSO_001,4.0,NaN
2,audiencia,colectivo,inferido,2,DISCURSO_001_FR_003,DISCURSO_001,4.0,NaN
3,pingüino,humano_individual,inferido,3,DISCURSO_001_FR_004,DISCURSO_001,NaN,1.0
4,Javier Milei,humano_individual,explícito,4,DISCURSO_001_FR_005,DISCURSO_001,NaN,NaN
5,La derecha,colectivo,explícito,4,DISCURSO_001_FR_005,DISCURSO_001,NaN,NaN
6,La población argentina en general,colectivo,inferido,4,DISCURSO_001_FR_005,DISCURSO_001,NaN,1.0
7,Agustín Laje,humano_individual,explícito,5,DISCURSO_001_FR_006,DISCURSO_001,NaN,1.0
8,El equipo de La Derecha Diario,colectivo,explícito,5,DISCURSO_001_FR_006,DISCURSO_001,NaN,1.0
9,Javier Milei,humano_individual,explícito,6,DISCURSO_001_FR_007,DISCURSO_001,NaN,NaN


In [1]:
# 3. Reprocesamiento de errores HAY QUE REVISARLA DE NUEVO POR INCOMPATIBILIDADES POSIBLES

import pandas as pd
import modulos.paths as paths
import modulos.prompts as prompts
from modulos.identificacion_actores import reprocesar_errores_identificacion
from modulos.modelo import llm

df_enunc = pd.read_csv(paths.discursos_enunc, encoding="utf-8-sig")
df_recortes = pd.read_csv(paths.recortes_prueba, encoding="utf-8-sig")

reprocesar_errores_identificacion(
    df_recortes=df_recortes,
    df_enunc=df_enunc,
    path_errores=paths.errores_identificacion_actores,
    path_salida=paths.errores_persistentes,
    prompt_template=prompts.PROMPT_IDENTIFICAR_ACTORES,
    intento=1,
    evitar_duplicados=True,
    modelo_llm=llm,
    mostrar_prompts=False
)

C:\ProgramData\anaconda3\envs\ag_env2\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


🔁 Reprocesando 1 errores...


C:\PROYECTOS\Primer MVP1\modulos\identificacion_actores.py:98: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  respuesta = modelo_llm([HumanMessage(content=prompt)]).content


✅ Agregados 1 nuevos registros a 'C:\PROYECTOS\Primer MVP1\errores\errores_persistentes.jsonl'
✅ Todos los errores fueron corregidos.


In [2]:
df_actores = pd.read_csv("data/C1. actores_identificados.csv", encoding="utf-8-sig")

df_actores.head(30)

,actor,tipo,modo,frase_idx,recorte_id,codigo,regla_de_inferencia,regla de inferencia
0,audiencia,colectivo,explícito,0,DISCURSO_001_FR_001,DISCURSO_001,NaN,NaN
1,audiencia,colectivo,inferido,1,DISCURSO_001_FR_002,DISCURSO_001,4.0,NaN
2,audiencia,colectivo,inferido,2,DISCURSO_001_FR_003,DISCURSO_001,4.0,NaN
3,pingüino,humano_individual,inferido,3,DISCURSO_001_FR_004,DISCURSO_001,NaN,1.0
4,Javier Milei,humano_individual,explícito,4,DISCURSO_001_FR_005,DISCURSO_001,NaN,NaN
5,La derecha,colectivo,explícito,4,DISCURSO_001_FR_005,DISCURSO_001,NaN,NaN
6,La población argentina en general,colectivo,inferido,4,DISCURSO_001_FR_005,DISCURSO_001,NaN,1.0
7,Agustín Laje,humano_individual,explícito,5,DISCURSO_001_FR_006,DISCURSO_001,NaN,1.0
8,El equipo de La Derecha Diario,colectivo,explícito,5,DISCURSO_001_FR_006,DISCURSO_001,NaN,1.0
9,Javier Milei,humano_individual,explícito,6,DISCURSO_001_FR_007,DISCURSO_001,NaN,NaN


In [1]:
# 4. Postprocesamiento

from modulos.identificacion_actores import validacion_actores

path_df_actores = "data/actores_identificados.csv"
path_df_recortes = "data/recortes_preprocesado_10.csv"
path_salida_validos = "data/actores_validos.csv"
path_salida_excluidos = "data/actores_excluidos.csv"

validacion_actores(
    path_df_actores=path_df_actores,
    path_df_recortes=path_df_recortes,
    path_salida_validos=path_salida_validos,
    path_salida_excluidos=path_salida_excluidos
)

C:\ProgramData\anaconda3\envs\ag_env2\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
Validando actores recorte DISCURSO_001_FR_001:   0%|                                             | 0/1 [00:00<?, ?it/s]C:\PROYECTOS\Primer MVP1\modulos\identificacion_actores.py:232: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  respuesta = llm([HumanMessage(content=prompt)]).content

Procesando recortes:  10%|██████▏                                                       | 1/10 [00:11<01:43, 11.48s/it]

DISCURSO_001_FR_001 - todos: ✅ válido



Validando actores recorte DISCURSO_001_FR_002:  33%|████████████▎                        | 1/3 [00:04<00:09,  4.69s/it]

DISCURSO_001_FR_002 - Carlos Pellegrini: ✅ válido



Validando actores recorte DISCURSO_001_FR_002:  67%|████████████████████████▋            | 2/3 [00:08<00:03,  3.94s/it]

...

DISCURSO_001_FR_009 - gente: ✅ válido



Validando actores recorte DISCURSO_001_FR_010:  50%|██████████████████▌                  | 1/2 [00:03<00:03,  3.38s/it]

DISCURSO_001_FR_010 - Pellegrini: ✅ válido



Procesando recortes: 100%|█████████████████████████████████████████████████████████████| 10/10 [01:35<00:00,  9.58s/it]

DISCURSO_001_FR_010 - la historia: ❌ excluido
✅ Guardados 17 actores validados en 'data/actores_validos.csv'
📄 Guardados 7 actores excluidos en 'data/actores_excluidos.csv'
